In [1]:
import mysql.connector
import pandas as pd
from datetime import date
from datetime import timedelta

conn = mysql.connector.connect(
    host = '10.139.132.109',
    user = 'od.shiva.g',
    password = 'SH$vIA19',
    auth_plugin = 'caching_sha2_password'
)

In [ ]:
## Getting the registered users for a the given date

cursor = conn.cursor()
query = """
        SELECT id,date_format(CONVERT_TZ(creationTime,'+00:00','+05:30'), '%Y-%m-%d') as createdOn FROM yp.yp_user
            where date_format(CONVERT_TZ(creationTime,'+00:00','+05:30'), '%Y-%m-%d') between '2019-04-01' and '2019-04-30';
        """
cursor.execute(query)
res = cursor.fetchall()
reg_df = pd.DataFrame()
reg_df = reg_df.append(res)
reg_df.columns = ['uid', 'createdOn']
reg_df.drop_duplicates('uid', inplace=True)

In [ ]:
len(reg_df)

In [ ]:
## Getting latest af_ids for given user_id

cursor = conn.cursor()
uids = list(reg_df['uid'].values)
query = """
           select userId,min(vendorId) as af_id from yp.yp_appsflyer_clevertap_id_history 
where type = "appsflyer" and userId in {}
group by userId;
            """
query = query.format(tuple(uids))
cursor.execute(query)
res = cursor.fetchall()

afid_uid_df = pd.DataFrame()
afid_uid_df = afid_uid_df.append(res)
afid_uid_df.columns = ['userId', 'af_id']

In [ ]:
x = reg_df.merge(afid_uid_df, how='left', left_on='uid', right_on='userId')
x['af_id'].fillna('AF_ID_NOT_FOUND', inplace=True)

In [ ]:
len(x)

In [ ]:
## Getting media_source for a given af_id

cursor = conn.cursor()    
af_ids = x['af_id'].values

query = """
           SELECT appsflyer_device_id, media_source, agency FROM  yp.yp_rf_appsflyer
where appsflyer_device_id in {};
            """
query = query.format(tuple(af_ids))
cursor.execute(query)
res = cursor.fetchall()

af_id_media_df = pd.DataFrame()
af_id_media_df = af_id_media_df.append(res)
af_id_media_df.columns = ['af_id', 'media_source', 'agency']

In [ ]:
y = x.merge(af_id_media_df, how='left', on='af_id')
y['media_source'].fillna('UnKnown_Media', inplace=True)
y1=y.drop_duplicates('uid')
y1=pd.DataFrame(y1)

In [ ]:
len(y1)

In [ ]:
## For getting eligible customers
cursor = conn.cursor()    
elig_ids = y1['uid'].values

query = """
        select uid,(case
when state = 33 then "yes"
else "no"
end) as eligible from yp.yp_user_state
WHERE uid IN {} and state = 33;
            """
query = query.format(tuple(elig_ids))
cursor.execute(query)
res = cursor.fetchall()

elig_ids_df = pd.DataFrame()
elig_ids_df = elig_ids_df.append(res)
elig_ids_df.columns = ['id', 'isEligible']

In [ ]:
elig_ids_df=elig_ids_df.drop_duplicates(subset='id',keep='first')

In [ ]:
len(elig_ids_df)

In [ ]:
## For getting confirmed customers
cursor = conn.cursor()    
con_ids = y1['uid'].values

query = """
        select uid,(case
when state = 53 && subState = 79 then "yes"
else "no"
end) as confirmed from yp.yp_user_state
where uid in {} and state = 53 and subState = 79;
            """
query = query.format(tuple(con_ids))
cursor.execute(query)
res = cursor.fetchall()

con_ids_df = pd.DataFrame()
con_ids_df = con_ids_df.append(res)
con_ids_df.columns = ['id', 'isConfirmed']

In [ ]:
con_ids_df=con_ids_df.drop_duplicates(subset='id',keep='first')

In [ ]:
len(con_ids_df)

In [ ]:
## For first loans
cursor = conn.cursor()    
first_loans_ids = y1['uid'].values

query = """
        select userid,count(disbursed_date) as first_loans from ypdynamic.yp_emi_data_tbl_v3
where loan_number = 1 and userid IN {}
group by userid;

            """
query = query.format(tuple(first_loans_ids))
cursor.execute(query)
res = cursor.fetchall()

first_loans_ids_df = pd.DataFrame()
first_loans_ids_df = first_loans_ids_df.append(res)
first_loans_ids_df.columns = ['id', 'first_loans']

In [ ]:
first_loans_ids_df = first_loans_ids_df.drop_duplicates(subset ='id',keep='first')

In [ ]:
len(first_loans_ids_df)

In [ ]:
## For total loans
cursor = conn.cursor()    
total_loans_ids = y1['uid'].values

query = """
        select userId, count(disbursedOn) as total_loans from yp.yp_loan
        where userId in {} and disbursedOn is not null group by userId;
            """
query = query.format(tuple(total_loans_ids))
cursor.execute(query)
res = cursor.fetchall()

total_loans_ids_df = pd.DataFrame()
total_loans_ids_df = total_loans_ids_df.append(res)
total_loans_ids_df.columns = ['id', 'total_loans']

In [ ]:
total_loans_ids_df = total_loans_ids_df.drop_duplicates(subset='id',keep='first')

In [ ]:
len(total_loans_ids_df)

In [ ]:
## For payment history of  loans
cursor = conn.cursor()    
payment_ids = first_loans_ids_df['id'].values

query = """
        select userid,emi_due_date,emi_paid_date from ypdynamic.yp_emi_data_tbl_v3 
where userid IN {} 
group by userid;

            """
query = query.format(tuple(payment_ids))
cursor.execute(query)
res = cursor.fetchall()

payment_ids_df = pd.DataFrame()
payment_ids_df = payment_ids_df.append(res)
payment_ids_df.columns = ['id', 'emi_due_date','emi_paid_date']

In [ ]:
len(payment_ids_df)

In [ ]:
payment_ids_df = payment_ids_df.drop_duplicates(subset='id',keep='first')

In [ ]:
len(payment_ids_df)

In [ ]:
payment_ids_df.to_csv('PaymentHistoryApril.csv',index=False)

In [ ]:
emi = pd.read_csv('PaymentHistoryApril.csv')

In [ ]:
emi.head()

In [ ]:
len(emi)

In [ ]:
emi['id'].count()

In [ ]:
final = y1.merge(elig_ids_df,how='left',left_on='uid',right_on='id').merge(con_ids_df, how='left', left_on='uid', right_on='id').merge(first_loans_ids_df, how='left', left_on='uid', right_on='id').merge(total_loans_ids_df, how='left', left_on='uid', right_on='id').merge(emi,how='left',left_on='uid',right_on='id')

In [ ]:
final.shape

In [ ]:
final.head()

In [ ]:
final['af_id'].value_counts().sum()

In [2]:
fraudapril = pd.read_csv('C:/Users/ShivaGSK/Desktop/April-July Fraud Data/Fraud April.csv',usecols=['transaction_Id','FRAUD_CATEGORY'])

In [3]:
fraudapril.shape

(3629241, 2)

In [4]:
fraudapril = fraudapril.drop_duplicates(subset='transaction_Id',keep='first')

In [5]:
fraudapril['FRAUD_CATEGORY'].isnull().sum()

0

In [6]:
final1=final.merge(fraudapril,how='left',left_on='af_id',right_on='transaction_Id')

NameError: name 'final' is not defined

In [ ]:
final1.head()

In [ ]:
final1['af_id'].isnull().sum()

In [ ]:
final1['FRAUD_CATEGORY'].isnull().sum()

In [ ]:
final1['FRAUD_CATEGORY'].value_counts()

In [ ]:
import numpy as np
final1['FRAUD_CATEGORY'] = final1.FRAUD_CATEGORY.replace(np.nan,'NotFound',regex = True)

In [ ]:
final1=pd.DataFrame(final1)

In [ ]:
final1[final1['af_id']=='AF_ID_NOT_FOUND']

In [ ]:
final1['FRAUD_CATEGORY'].isnull().sum()

In [ ]:
final1.isnull().sum()

In [ ]:
final1.shape

In [ ]:
final1['FRAUD_CATEGORY'].value_counts()

In [ ]:
final1.to_csv('FinalApril.csv',index=False)

In [46]:
import pandas as pd
overall = pd.read_csv('C:/Users/ShivaGSK/Desktop/April/FinalAprill.csv')

In [47]:
overall.head()

,uid,emipaid_date,createdOn,userId,af_id,media_source,agency,id_x,isEligible,id_y,...,emi_due_date,emi_paid_date,Day Difference,Delayed Payment,Prepayment,Delayed Days,Prepaid Days,FRAUD_CATEGORY,transaction_Id,Not Paid
0,5047503,NaN,01-04-2019,5047503.0,1554056890197-1995931354730990750,Facebook Ads,fbmagic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056890197-1995931354730990750,NaN
1,5047504,NaN,01-04-2019,5047504.0,1554056938804-829383679934845534,googleadwords_int,NaN,5047504.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056938804-829383679934845534,NaN
2,5047505,NaN,01-04-2019,5047505.0,1554056795618-3213275846379819322,googleadwords_int,NaN,5047505.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056795618-3213275846379819322,NaN
3,5047506,NaN,01-04-2019,5047506.0,1554056633711-3560512945014479334,googleadwords_int,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056633711-3560512945014479334,NaN
4,5047507,NaN,01-04-2019,5047507.0,1554056900284-763085451673634428,Facebook Ads,fbmagic,5047507.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056900284-763085451673634428,NaN


In [48]:
overall.isnull().sum()

uid                      0
emipaid_date        828809
createdOn                0
userId                2050
af_id                    0
media_source          2360
agency              710662
id_x                553400
isEligible          553400
id_y                756861
isConfirmed         756861
id_x.1              828809
first_loans         828809
id_y.1              828780
total_loans         828780
id                  828809
emi_due_date        828809
emi_paid_date       925099
Day Difference      828809
Delayed Payment     920090
Prepayment          890807
Delayed Days        920090
Prepaid Days        890807
FRAUD_CATEGORY           0
transaction_Id      236206
Not Paid            950044
dtype: int64

In [49]:
overall['media_source'].fillna('UnKnown_Media', inplace=True)

In [50]:
overall.head()

,uid,emipaid_date,createdOn,userId,af_id,media_source,agency,id_x,isEligible,id_y,...,emi_due_date,emi_paid_date,Day Difference,Delayed Payment,Prepayment,Delayed Days,Prepaid Days,FRAUD_CATEGORY,transaction_Id,Not Paid
0,5047503,NaN,01-04-2019,5047503.0,1554056890197-1995931354730990750,Facebook Ads,fbmagic,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056890197-1995931354730990750,NaN
1,5047504,NaN,01-04-2019,5047504.0,1554056938804-829383679934845534,googleadwords_int,NaN,5047504.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056938804-829383679934845534,NaN
2,5047505,NaN,01-04-2019,5047505.0,1554056795618-3213275846379819322,googleadwords_int,NaN,5047505.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056795618-3213275846379819322,NaN
3,5047506,NaN,01-04-2019,5047506.0,1554056633711-3560512945014479334,googleadwords_int,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056633711-3560512945014479334,NaN
4,5047507,NaN,01-04-2019,5047507.0,1554056900284-763085451673634428,Facebook Ads,fbmagic,5047507.0,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOT_FRAUD,1554056900284-763085451673634428,NaN


In [51]:
overall.shape

(982088, 26)

In [52]:
len(overall['af_id'])

982088

In [53]:
overall.isnull().sum()

uid                      0
emipaid_date        828809
createdOn                0
userId                2050
af_id                    0
media_source             0
agency              710662
id_x                553400
isEligible          553400
id_y                756861
isConfirmed         756861
id_x.1              828809
first_loans         828809
id_y.1              828780
total_loans         828780
id                  828809
emi_due_date        828809
emi_paid_date       925099
Day Difference      828809
Delayed Payment     920090
Prepayment          890807
Delayed Days        920090
Prepaid Days        890807
FRAUD_CATEGORY           0
transaction_Id      236206
Not Paid            950044
dtype: int64

In [54]:
# x = pd.pivot_table(overall, index='FRAUD_CATEGORY', values=['af_id', 'isEligible','isConfirmed', 'first_loans', 'total_loans'], aggfunc={'af_id' : [lambda x : len(x)], 'isEligible' : [lambda x : pd.notnull(x).sum()],'isConfirmed' : [lambda x : pd.notnull(x).sum()],'first_loans' : [lambda x : pd.notnull(x).sum()],'total_loans' : [lambda x : pd.notnull(x).sum()]})
# x

In [55]:
Overall = overall.groupby(['FRAUD_CATEGORY']).count()
Overall=Overall[['af_id','isEligible','isConfirmed','first_loans','Delayed Payment ','Prepayment','Not Paid']]
Overall

,af_id,isEligible,isConfirmed,first_loans,Delayed Payment,Prepayment,Not Paid
FRAUD_CATEGORY,,,,,,,
BLACKLISTED_IP,2,1,0,0,0,0,0
Browser,17,9,4,3,1,2,1
CLICK UNDER,1110,181,50,31,10,21,6
Click Spam,22805,15513,9010,6437,2527,3910,1311
DUPLICATE_USER,16557,8604,4867,3462,1239,2223,636
Device,2096,1286,753,517,225,292,99
FAKE_ATRIBUTION,135473,61861,32150,21151,8892,12259,4585
FAKE_DEVICE,24349,11225,6131,4122,1697,2425,876
INCORRECT_REGION,3,2,0,0,0,0,0


In [56]:
Overall = Overall.rename(columns={'af_id':'Registrations','isEligible':'Eligible','isConfirmed':'Confirmed','first_loans':'FirstLoans'})
Overall

,Registrations,Eligible,Confirmed,FirstLoans,Delayed Payment,Prepayment,Not Paid
FRAUD_CATEGORY,,,,,,,
BLACKLISTED_IP,2,1,0,0,0,0,0
Browser,17,9,4,3,1,2,1
CLICK UNDER,1110,181,50,31,10,21,6
Click Spam,22805,15513,9010,6437,2527,3910,1311
DUPLICATE_USER,16557,8604,4867,3462,1239,2223,636
Device,2096,1286,753,517,225,292,99
FAKE_ATRIBUTION,135473,61861,32150,21151,8892,12259,4585
FAKE_DEVICE,24349,11225,6131,4122,1697,2425,876
INCORRECT_REGION,3,2,0,0,0,0,0


In [57]:
Overall['FirstLoanConversions'] = Overall['FirstLoans']/Overall['Registrations']
Overall['Delayed Payment %'] = Overall['Delayed Payment ']/Overall['FirstLoans']
Overall['Prepayment %'] = Overall['Prepayment']/Overall['FirstLoans']
Overall['Not Paid %'] = Overall['Not Paid']/Overall['FirstLoans']

In [58]:
Overall

,Registrations,Eligible,Confirmed,FirstLoans,Delayed Payment,Prepayment,Not Paid,FirstLoanConversions,Delayed Payment %,Prepayment %,Not Paid %
FRAUD_CATEGORY,,,,,,,,,,,
BLACKLISTED_IP,2,1,0,0,0,0,0,0.000000,NaN,NaN,NaN
Browser,17,9,4,3,1,2,1,0.176471,0.333333,0.666667,0.333333
CLICK UNDER,1110,181,50,31,10,21,6,0.027928,0.322581,0.677419,0.193548
Click Spam,22805,15513,9010,6437,2527,3910,1311,0.282263,0.392574,0.607426,0.203666
DUPLICATE_USER,16557,8604,4867,3462,1239,2223,636,0.209096,0.357886,0.642114,0.183709
Device,2096,1286,753,517,225,292,99,0.246660,0.435203,0.564797,0.191489
FAKE_ATRIBUTION,135473,61861,32150,21151,8892,12259,4585,0.156127,0.420406,0.579594,0.216775
FAKE_DEVICE,24349,11225,6131,4122,1697,2425,876,0.169288,0.411693,0.588307,0.212518
INCORRECT_REGION,3,2,0,0,0,0,0,0.000000,NaN,NaN,NaN


In [60]:
Overall.to_csv('OverallAprilNotPaid.csv')

In [61]:
channel = overall.groupby(['media_source']).count()

In [62]:
channel=channel[['af_id','isEligible','isConfirmed','first_loans','Delayed Payment ','Prepayment','Not Paid']]

In [65]:
channel

,af_id,isEligible,isConfirmed,first_loans,Delayed Payment,Prepayment,Not Paid
media_source,,,,,,,
Airtel,17,7,2,1,0,1,0
Credit Mantri,6,5,3,2,2,0,1
Facebook Ads,269065,98267,43225,26318,10080,16238,5161
Mi,1105,577,263,156,47,109,21
Mi Android,1,0,0,0,0,0,0
None,48,37,21,17,4,13,0
Organic,77816,49584,28496,19946,8912,11034,4528
Paisa Bazaar,1472,1018,553,336,96,240,42
Referral,33955,25221,15316,11239,4096,7143,1838


In [66]:
channel = channel.rename(columns={'af_id':'Registrations','isEligible':'Eligible','isConfirmed':'Confirmed','first_loans':'FirstLoans'})
channel

,Registrations,Eligible,Confirmed,FirstLoans,Delayed Payment,Prepayment,Not Paid
media_source,,,,,,,
Airtel,17,7,2,1,0,1,0
Credit Mantri,6,5,3,2,2,0,1
Facebook Ads,269065,98267,43225,26318,10080,16238,5161
Mi,1105,577,263,156,47,109,21
Mi Android,1,0,0,0,0,0,0
None,48,37,21,17,4,13,0
Organic,77816,49584,28496,19946,8912,11034,4528
Paisa Bazaar,1472,1018,553,336,96,240,42
Referral,33955,25221,15316,11239,4096,7143,1838


In [67]:
channel['FirstLoanConversions'] = channel['FirstLoans']/channel['Registrations']
channel['Delayed Payment %'] = channel['Delayed Payment ']/channel['FirstLoans']
channel['Prepayment %'] = channel['Prepayment']/channel['FirstLoans']
channel['Not Paid %'] = channel['Not Paid']/channel['FirstLoans']

In [68]:
channel

,Registrations,Eligible,Confirmed,FirstLoans,Delayed Payment,Prepayment,Not Paid,FirstLoanConversions,Delayed Payment %,Prepayment %,Not Paid %
media_source,,,,,,,,,,,
Airtel,17,7,2,1,0,1,0,0.058824,0.000000,1.000000,0.000000
Credit Mantri,6,5,3,2,2,0,1,0.333333,1.000000,0.000000,0.500000
Facebook Ads,269065,98267,43225,26318,10080,16238,5161,0.097813,0.383008,0.616992,0.196102
Mi,1105,577,263,156,47,109,21,0.141176,0.301282,0.698718,0.134615
Mi Android,1,0,0,0,0,0,0,0.000000,NaN,NaN,NaN
None,48,37,21,17,4,13,0,0.354167,0.235294,0.764706,0.000000
Organic,77816,49584,28496,19946,8912,11034,4528,0.256323,0.446806,0.553194,0.227013
Paisa Bazaar,1472,1018,553,336,96,240,42,0.228261,0.285714,0.714286,0.125000
Referral,33955,25221,15316,11239,4096,7143,1838,0.330997,0.364445,0.635555,0.163538


In [69]:
channel.to_csv('ChannelwiseAprilNotPaid.csv')

In [70]:
channel = overall.groupby(['media_source','FRAUD_CATEGORY']).count()

In [71]:
channel=channel[['af_id','isEligible','isConfirmed','first_loans','Delayed Payment ','Prepayment','Not Paid']]

In [72]:
channel

af_id  isEligible  isConfirmed  first_loans  \
media_source  FRAUD_CATEGORY                                                   
Airtel        NOT_FRAUD              7           4            2            1   
              NotFound              10           3            0            0   
Credit Mantri FAKE_DEVICE            1           1            0            0   
              NOT_FRAUD              5           4            3            2   
Facebook Ads  NOT_FRAUD         265539       96853        42508        25809   
              NotFound            3526        1414          717          509   
Mi            DUPLICATE_USER        30          18            9            4   
              FAKE_ATRIBUTION        3           1            0            0   
              FAKE_DEVICE          201          98           37           24   
              NON_PLAY_STORE        19           9            7            5   
              NOT_FRAUD            843         448          207          120   
              NotFound               4           0            0            0   
              VIRTUAL_NETWORK        5           3            3            3   
Mi Android    NOT_FRAUD              1           0            0            0   
None          NOT_FRAUD             47          37           21           17   
              NotFound               1           0            0            0   
Organic       NOT_FRAUD          75133       48543        27854        19461   
              NotFound            2683        1041          642          485   
Paisa Bazaar  NOT_FRAUD           1299         936          504          300   
              NotFound             173          82           49           36   
Referral      NOT_FRAUD          33005       24760        15038        11035   
              NotFound             950         461          278          204   
UnKnown_Media Browser                1           0            0            0   
              Click Spam           109          72           41           31   
              DUPLICATE_USER       233         122           66           39   
              FAKE_ATRIBUTION      135          72           40           29   
              FAKE_DEVICE          273         104           46           27   
              INCORRECT_REGION       1           1            0            0   
              NON_PLAY_STORE        34          15            8            3   
              NOT_FRAUD           1767         795          367          236   
...                                ...         ...          ...          ...   
taptica_int   CLICK UNDER            3           1            1            0   
              Click Spam          3309        2332         1368          970   
              DUPLICATE_USER      2228        1342          786          558   
              Device               479         274          162          114   
              FAKE_ATRIBUTION    24842       12623         6698         4400   
              FAKE_DEVICE         2242        1041          540          380   
              ISP                   13          10            6            6   
              NON_PLAY_STORE      1626         683          342          236   
              NOT_FRAUD          10405        5489         2935         1960   
              NotFound            1571         680          390          294   
              OS                   207         126           55           34   
              VIRTUAL_NETWORK      129          78           49           30   
unityads_int  Click Spam             8           3            1            1   
              DUPLICATE_USER         4           2            0            0   
              FAKE_ATRIBUTION        2           0            0            0   
              FAKE_DEVICE            2           0            0            0   
              NON_PLAY_STORE         2           1            1            0   
              NOT_FRAUD          

In [73]:
channel = channel.rename(columns={'af_id':'Registrations','isEligible':'Eligible','isConfirmed':'Confirmed','first_loans':'FirstLoans'})
channel

Registrations  Eligible  Confirmed  \
media_source  FRAUD_CATEGORY                                         
Airtel        NOT_FRAUD                     7         4          2   
              NotFound                     10         3          0   
Credit Mantri FAKE_DEVICE                   1         1          0   
              NOT_FRAUD                     5         4          3   
Facebook Ads  NOT_FRAUD                265539     96853      42508   
              NotFound                   3526      1414        717   
Mi            DUPLICATE_USER               30        18          9   
              FAKE_ATRIBUTION               3         1          0   
              FAKE_DEVICE                 201        98         37   
              NON_PLAY_STORE               19         9          7   
              NOT_FRAUD                   843       448        207   
              NotFound                      4         0          0   
              VIRTUAL_NETWORK               5         3          3   
Mi Android    NOT_FRAUD                     1         0          0   
None          NOT_FRAUD                    47        37         21   
              NotFound                      1         0          0   
Organic       NOT_FRAUD                 75133     48543      27854   
              NotFound                   2683      1041        642   
Paisa Bazaar  NOT_FRAUD                  1299       936        504   
              NotFound                    173        82         49   
Referral      NOT_FRAUD                 33005     24760      15038   
              NotFound                    950       461        278   
UnKnown_Media Browser                       1         0          0   
              Click Spam                  109        72         41   
              DUPLICATE_USER              233       122         66   
              FAKE_ATRIBUTION             135        72         40   
              FAKE_DEVICE                 273       104         46   
              INCORRECT_REGION              1         1          0   
              NON_PLAY_STORE               34        15          8   
              NOT_FRAUD                  1767       795        367   
...                                       ...       ...        ...   
taptica_int   CLICK UNDER                   3         1          1   
              Click Spam                 3309      2332       1368   
              DUPLICATE_USER             2228      1342        786   
              Device                      479       274        162   
              FAKE_ATRIBUTION           24842     12623       6698   
              FAKE_DEVICE                2242      1041        540   
              ISP                          13        10          6   
              NON_PLAY_STORE             1626       683        342   
              NOT_FRAUD                 10405      5489       2935   
              NotFound                   1571       680        390   
              OS                          207       126         55   
              VIRTUAL_NETWORK             129        78         49   
unityads_int  Click Spam                    8         3          1   
              DUPLICATE_USER                4         2          0   
              FAKE_ATRIBUTION               2         0          0   
              FAKE_DEVICE                   2         0          0   
              NON_PLAY_STORE                2         1          1   
              NOT_FRAUD                    41        19          4   
              NotFound                     35        15         11   
yeahmobi_int  Browser                       5         4          2   
              Click Spam                 1047       718        392   
              DUPLICATE_USER             2066       910        470   
              Device                      219       145         91   
              FAKE_ATRIBUTION           31611     14163       7430   
              FAKE_DEVICE                3

In [74]:
channel['FirstLoanConversions'] = channel['FirstLoans']/channel['Registrations']
channel['Delayed Payment %'] = channel['Delayed Payment ']/channel['FirstLoans']
channel['Prepayment %'] = channel['Prepayment']/channel['FirstLoans']
channel['Not Paid %'] = channel['Not Paid']/channel['FirstLoans']

In [75]:
channel

Registrations  Eligible  Confirmed  \
media_source  FRAUD_CATEGORY                                         
Airtel        NOT_FRAUD                     7         4          2   
              NotFound                     10         3          0   
Credit Mantri FAKE_DEVICE                   1         1          0   
              NOT_FRAUD                     5         4          3   
Facebook Ads  NOT_FRAUD                265539     96853      42508   
              NotFound                   3526      1414        717   
Mi            DUPLICATE_USER               30        18          9   
              FAKE_ATRIBUTION               3         1          0   
              FAKE_DEVICE                 201        98         37   
              NON_PLAY_STORE               19         9          7   
              NOT_FRAUD                   843       448        207   
              NotFound                      4         0          0   
              VIRTUAL_NETWORK               5         3          3   
Mi Android    NOT_FRAUD                     1         0          0   
None          NOT_FRAUD                    47        37         21   
              NotFound                      1         0          0   
Organic       NOT_FRAUD                 75133     48543      27854   
              NotFound                   2683      1041        642   
Paisa Bazaar  NOT_FRAUD                  1299       936        504   
              NotFound                    173        82         49   
Referral      NOT_FRAUD                 33005     24760      15038   
              NotFound                    950       461        278   
UnKnown_Media Browser                       1         0          0   
              Click Spam                  109        72         41   
              DUPLICATE_USER              233       122         66   
              FAKE_ATRIBUTION             135        72         40   
              FAKE_DEVICE                 273       104         46   
              INCORRECT_REGION              1         1          0   
              NON_PLAY_STORE               34        15          8   
              NOT_FRAUD                  1767       795        367   
...                                       ...       ...        ...   
taptica_int   CLICK UNDER                   3         1          1   
              Click Spam                 3309      2332       1368   
              DUPLICATE_USER             2228      1342        786   
              Device                      479       274        162   
              FAKE_ATRIBUTION           24842     12623       6698   
              FAKE_DEVICE                2242      1041        540   
              ISP                          13        10          6   
              NON_PLAY_STORE             1626       683        342   
              NOT_FRAUD                 10405      5489       2935   
              NotFound                   1571       680        390   
              OS                          207       126         55   
              VIRTUAL_NETWORK             129        78         49   
unityads_int  Click Spam                    8         3          1   
              DUPLICATE_USER                4         2          0   
              FAKE_ATRIBUTION               2         0          0   
              FAKE_DEVICE                   2         0          0   
              NON_PLAY_STORE                2         1          1   
              NOT_FRAUD                    41        19          4   
              NotFound                     35        15         11   
yeahmobi_int  Browser                       5         4          2   
              Click Spam                 1047       718        392   
              DUPLICATE_USER             2066       910        470   
              Device                      219       145         91   
              FAKE_ATRIBUTION           31611     14163       7430   
              FAKE_DEVICE                3

In [76]:
channel.to_csv('ChannelwiseFraudAprilNotPaid.csv')

In [ ]:
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl import Workbook
# from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
# from openpyxl.styles import colors
# from openpyxl.cell import Cell
# from openpyxl.utils import get_column_letter, column_index_from_string


# final=pd.read_csv("FinalMay.csv")
# # final.drop(columns='Unnamed: 0', inplace=True)
# final['media_source'].fillna('UnKnown_Media', inplace=True)
# # def new_loans(x):
# #     return x.notnull().sum()

# wb = Workbook()
# for ms in ['Aggregate',"UnKnown_Media",
# "Facebook Ads",
# "mobvista_int",
# "googleadwords_int",
# "inmobi_int",
# "Organic",
# "yeahmobi_int",
# "Referral",
# "mobpower_int",
# "novabeyond_int",
# "Paisa Bazaar",
# "taptica_int",
# "mfaas_int",
# "appsflowerprime_int",
# "aliothmob_int",
# "Mi",
# "xiaomiglobal_int",
# "Yelo",
# "affle_int",
# "TrustChekr",
# "None",
# "applift_int",
# "trustchekr",
# "appliftcustom_int",
# "mobisummer_int",
# "leadbolt_int",
# "unityads_int",
# "appnext_int",
# "seventyninemobi_int",
# "Airtel",
# "candyadzcake_int",
# "adapptmobi_int",
# "indusos_int",
# "Credit Mantri",
# "mobavenue_int",
# "paisabazaar_int",
# "salesdoubler_int",
# "lendingadda360_int",
# ]:
#     if ms == 'Aggregate':
#         temp = final
#     else:
#         temp = final[final['media_source'] == ms]
    
#     x = pd.pivot_table(temp, index='FRAUD_CATEGORY', values=['af_id', 'isEligible','isConfirmed', 'first_loans','emi_due_date','emi_paid_date','Not Paid','Delayed','Prepaid','OnDate'], aggfunc={'af_id' : [lambda x : pd.notnull(x).sum()], 'isEligible' : [lambda x : pd.notnull(x).sum()],'isConfirmed' : [lambda x : pd.notnull(x).sum()],'first_loans' : [lambda x : pd.notnull(x).sum()],'emi_due_date' : [lambda x : pd.notnull(x).sum()],'emi_paid_date' : [lambda x : pd.notnull(x).sum()],'Not Paid' : [lambda x : pd.notnull(x).sum()],'Delayed' : [lambda x : pd.notnull(x).sum()],'Prepaid' : [lambda x : pd.notnull(x).sum()],'OnDate' : [lambda x : pd.notnull(x).sum()]})
#     x.columns = ['af_id', 'isEligible', 'isConfirmed','first_loans','emi_due_date','emi_paid_date','Not Paid','Delayed','Prepaid','OnDate']
#     x.rename(columns={"af_id":"Reg","isEligible":"Eligible","isConfirmed":"Confirmed","first_loans":"FirstLoans","emi_due_date":"Due Emis","emi_paid_date":"Paid Emis"}, inplace=True)
#     x=x[['Reg','Eligible','Confirmed','FirstLoans','Due Emis','Paid Emis','Not Paid','Delayed','Prepaid','OnDate']]
#     ws = wb.create_sheet(title=ms)
#     for r in dataframe_to_rows(x):
#         ws.append(r)

# wb.save("channelwise_Fraud_Funnel_FinalMay.xlsx")
